In [13]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

#options = EdgeOptions()
#options.use_chromium = True
driver = webdriver.Chrome()
url = "https://www.amazon.com"
driver.get(url)

In [11]:
def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_2'
    search_term = search_term.replace(' ', '+')
    return template.format(search_term)

url = get_url('instant ramen')
print(url)

https://www.amazon.com/s?k=instant+ramen&ref=nb_sb_noss_2


In [48]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [56]:
results = soup.find_all('div', {'data-component-type': 's-search-result'})
len(results)
results[0]

<div class="sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 AdHolder sg-col sg-col-4-of-20" data-asin="B08F7C14BG" data-cel-widget="search_result_1" data-component-id="8" data-component-type="s-search-result" data-index="1" data-uuid="836e4d2f-a501-4955-a764-675e437aa294"><div class="sg-col-inner">
<span cel_widget_id="MAIN-SEARCH_RESULTS-1" class="celwidget slot=MAIN template=SEARCH_RESULTS widgetId=search-results" data-cel-widget="MAIN-SEARCH_RESULTS-1" data-csa-c-id="123epe-yhqtpd-3dyqqv-8dcnxf">
<div class="rush-component s-expand-height" data-component-id="9" data-component-props='{"percentageShownToFire":"50","batchable":true,"requiredElementSelector":".s-image","url":"https://www.amazon.com/gp/sponsored-products/logging/log-action.html?qualifier=1607155731&amp;id=5579779478771567&amp;widgetName=sp_atf&amp;adId=200029952300261&amp;eventType=1&amp;adIndex=0"}' data-component-type="s-impression-logger">
<div class="rush-component s-expand-height" data-component-id="10" data-comp

In [58]:
item = results[0]
atag = item.h2.a
description = atag.text.strip()


'Sapporo Ichiban Variety Flavors Mix Packs Instant Ramen Noodles, Tonkotsu, Shio, Miso, and Yakisoba Flavors Noodle Soup (Pack of 24)'

In [101]:
import pandas as pd

def extract_record(item):
    # Description
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://amazon.com' + atag.get('href')
    
    try:
        # Price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return        
        
    try:    
        # Review and Ratings
        rating = item.i.text
        rating_int = float(rating[0:3])
        review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text
        review_count_int = int(review_count.replace(',',''))
    except AttributeError:
        rating = ''
        review_count = ''

    result = (url, description, price, rating_int, review_count_int)
    
    return result

records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

In [102]:
df = pd.DataFrame(records)
df.columns = ('url', 'description', 'price', 'rating', 'review_count')
df.head()

,url,description,price,rating,review_count
0,https://amazon.com/gp/slredirect/picassoRedire...,Sapporo Ichiban Variety Flavors Mix Packs Inst...,$31.49,4.8,24
1,https://amazon.com/Menraku-Variety-Tonkotsu-Se...,"Menraku Variety Mix Flavors, Shio, Miso, Tonko...",$42.99,4.2,56
2,https://amazon.com/Nissin-Demae-Garlic-Instant...,Nissin Demae Black Garlic Oil Instant Ramen No...,$26.98,4.6,950
3,https://amazon.com/Nissin-Raoh-Miso-3-77-Ounce...,"Nissin RAOH, Miso Flavor, Authentic Japanese-S...",$11.94,4.5,1631
4,https://amazon.com/Japanese-populer-ICHIRAN-in...,"Japanese populer Ramen ""ICHIRAN"" instant noodl...",$33.60,4.7,744


In [104]:
variables1 = ['url', 'description', 'price', 'rating', 'review_count']
newdf = df.loc[(df['review_count']>25),variables1]
newdf.to_csv('amazonramen.csv', index=False)

In [1]:
import csv
from bs4 import BeautifulSoup
import pandas as pd
from urllib import request

def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_2'
    search_term = search_term.replace(' ', '+')
    return template.format(search_term)

url = get_url('instant ramen')
response = request.urlopen(url)
html = response.read().decode('utf8')

soup = BeautifulSoup(html, 'html.parser')

def extract_record(item):
    # Description
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://amazon.com' + atag.get('href')
    
    try:
        # Price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return        
        
    try:    
        # Review and Ratings
        rating = item.i.text
        rating_int = float(rating[0:3])
        review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text
        review_count_int = int(review_count.replace(',',''))
    except AttributeError:
        rating = ''
        review_count = ''

    result = (url, description, price, rating_int, review_count_int)
    
    return result

records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    record = extract_record(item)
    if record:
        records.append(record)
        
amazondf = pd.DataFrame(records)
amazondf.columns = ('url', 'description', 'price', 'rating', 'review_count')
amazondf.head()

HTTPError: HTTP Error 503: Service Unavailable